In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to run all arguments and not just the last one

In [48]:
# Using IRIS dataset
# Limited to linear classifications
# Implimentation of perceptron learning in python
# The perceptron is an example of a single layer neural network
import numpy as np
class perceptron(object):
    """Perceptron classifier
    Parameters
    ----------
    eta: float
        Learning Rate (between 0 and 1)
    n_iter: int
        Passes over the training set
     
    Attributes
    ----------
    w_: id-array
        weights after filtering 
    errors_: list
        number of misclassifications in every epoch
    """
    def __init__(self, eta=0.1, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    def fit(self, X, y):
        """Fit training data
        Parameters
        ----------
        X: {array-like}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and 
            n_features is the number of features
        y: array-like, shape = [n_samples]
            Target Values
        returns
        -------
        self: object
        """
        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_ += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return(self)
    
    def net_input(self, X):
        """Calculate` net input"""
        return np.dot(X, self.w_[1:] + self.w_[0])
    
    def predict(self, X):
        """returns class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)


# using this perceptron implementation, we can now initialize new perceptron objects with a given learning rate eta and n_iter, 
# which is the number of epochs. note that numpy's arithmetic operations are vectorized

In [50]:
# Training a perceptron model on the iris dataset (using only 2 features and 2 flower classes) 
# The perceptron method can be extended to multiple classes using the one vs all technique (page 28 of the book)
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

URLError: <urlopen error Tunnel connection failed: 407 Proxy Authentication Required>

In [51]:
df.tail(6) # We extract the first 100 class labels that correspond to 50 iris-Setosa and 50 iris-Versicolor flowers
# and convert the class labels to int
import matplotlib.pyplot as plt

,0,1,2,3,4
144,6.7,3.3,5.7,2.5,Iris-virginica
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [52]:
y = df.iloc[0:100, 4].values
y = np.where(y == 'Iris-setosa', -1, 1)
X = df.iloc[0:100, [0, 2]].values
plt.scatter(X[:50, 0], X[:50, 1], color='red', marker='o', label='setosa')
plt.scatter(X[50:100, 0], X[50:100, 1], color='blue', marker='x', label='versicolor')
plt.xlabel('petal length')
plt.ylabel('sepal length')
plt.legend(loc='upper left')
plt.show()

In [19]:
# Time to train our perceptron algorithm on the iris dataset 
ppn = perceptron(eta=.1, n_iter=10)
ppn.fit(X, y)

In [22]:
plt.plot(range(1, len(ppn.errors_) + 1), ppn.errors_, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Number of updates')

plt.tight_layout()
plt.show()

In [53]:
# To plot a decision boundary
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, resolution=.02):
    
    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() -1, X[:, 0].max() + 1   # min and max value for our 2 features
    x2_min, x2_max = X[:, 1].min() -1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution), np.arange(x2_min, x2_max, resolution)) # Creating a pair of
    # grid arrays 
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T) # ravel creates a contiguous flattened array to create a matrix
    # with the same number of elements as iris. also, the classifier ppn has a predict function already defined by us 
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    # Plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1], alpha=.8, c=cmap(idx), marker=markers[idx], label=cl)

In [54]:
plot_decision_regions(X, Y, classifier=ppn)
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.legend(loc='upper left')

plt.tight_layout()
# plt.savefig('./perceptron_2.png', dpi=300)
plt.show()

# All data in the training set was perfectly classified

In [ ]:
# Adaptive linear neurons and the convergence of learning
# another single layer neural network is Adaline(Adaptive linear neuron) which is an improvement on the perceptron
# weights in adeline are updated based on a linear activation function instead of a step function
# Gradient descent can be used to optimize the objective function/minimize the cost function

In [55]:
# Implementing adaptive learning neuron
class AdalineGD(object):
    """Adaptive  linear neuron classifier
    Parameters
    ----------
    eta: float
        Learning rate
    n_iter: int
        passes over the training dataset
    
    Attributes
    ----------
    w_: 1d-array
        weights after fitting
    errors_: list
        number of misclassifications in every epoch
    """
    
    def __init__(self, eta=0.01, n_iter=50):
        self.eta = eta
        self.n_iter = n_iter
        
    def fit(self, X, y):
        """Fit training data
        Parameters
        ----------
        X: {array-like},  shape = [n_samples, n_features],
        training vectors where n_samples is the number of samples and n_features is the number of features
        y: array-like, shape = [n_samples]
        target values
        Returns
        self: object
        """
        self.w_ = np.zeros(1 + X.shape[1])
        self.cost_ = []
        for i in range(self.n_iter):
            output = self.net_input(X)
            errors = (y - output)
            self.w_[1:] += self.eta + X.T.dot(errors)
            self.w_[0] += self.eta + errors.sum()
            cost = (errors**2).sum()/2.0
            self.cost_.append(cost)
        return(self)
    
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]
    
    def activation(self, X):
        """Compute linear activation"""
        return self.net_input(X)
    
    def predict(self, X):
        """return class labels after unit step"""
        return np.where(self.activation(X) >= 0.0, 1, -1)   

# Instead of updating the weights after evaluating each individual training
# sample, as in the perceptron, we calculate the gradient based on the whole
# training dataset via self.eta * errors.sum() for the zero-weight and via
# self.eta * X.T.dot(errors) for the weights 1 to m where X.T.dot(errors)
# is a matrix-vector multiplication between our feature matrix and the error vector.
# Similar to the previous perceptron implementation, we collect the cost values in
# a list self.cost_ to check if the algorithm converged after training.

In [59]:
# Here, n_iter and learning rate are called hyperparameters which need to be optimized for best model performance
# Plotting the cost function against number of epochs
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

ada1 = AdalineGD(n_iter=10, eta=0.01).fit(X, y) # a learning rate that is too large can overshoot the global minimum
ax[0].plot(range(1, len(ada1.cost_) + 1), np.log10(ada1.cost_), marker='o')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('log(Sum-squared-error)')
ax[0].set_title('Adaline - Learning rate 0.01')

ada2 = AdalineGD(n_iter=10, eta=0.0001).fit(X, y)
ax[1].plot(range(1, len(ada2.cost_) + 1), ada2.cost_, marker='o')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Sum-squared-error')
ax[1].set_title('Adaline - Learning rate 0.0001')

plt.tight_layout()
# plt.savefig('./adaline_1.png', dpi=300)
plt.show()

In [60]:
# Many methods require feature scaling to perform better. 
# For gradient descent, we will try using standardization
X_std = np.copy(X)
X_std[:, 0] = (X[:, 0] - X[:, 0].mean())/X[:, 0].std()
X_std[:, 1] = (X[:, 1] - X[:, 1].mean())/X[:, 1].std()

In [61]:
ada = AdalineGD(n_iter=15, eta=0.1)
ada.fit(X_std, y) # This does converge for the author; but not for me

plot_decision_regions(X_std, y, classifier=ada)
plt.title('Adaline - Gradient Descent')
plt.xlabel('sepal length [standardized]')
plt.ylabel('petal length [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./adaline_2.png', dpi=300)
plt.show()

plt.plot(range(1, len(ada.cost_) + 1), ada.cost_, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Sum-squared-error')

plt.tight_layout()
# plt.savefig('./adaline_3.png', dpi=300)
plt.show()

In [ ]:
# The method above is called batch gradient descent; a computationally more effective alternative is called 
# stochastic gradient descent also called iterative or on-line gradient descent
# Instead of updating the weights on the sum of accumulated errors over all samples, we update the weights incrementally 
# for each training sample. For accuracy, it is important to present it with data in a random order so we shuffle the training
# data at every epoch
# Stochastic gradient descent can also be used for on-line learning (learning on the fly as new training data arrives)
# We only need to make a few adjustments to adeline gradient descent to modify the learning algorithm to to update the 
# weights via stochastic gradient descent

In [65]:
from numpy.random import seed
class AdalineSGD(object):
    """Adaptive  linear neuron classifier
    Parameters
    ----------
    eta: float
        Learning rate
    n_iter: int
        passes over the training dataset
    
    Attributes
    ----------
    w_: 1d-array
        weights after fitting
    errors_: list
        number of misclassifications in every epoch
    shuffle: bool(default: True)
    random_state: int(default: none)
    """
    
    def __init__(self, eta=0.01, n_iter=50, shuffle = True, random_state=None):
        self.eta = eta
        self.n_iter = n_iter
        self.w_initialized = False
        self.shuffle = False
        if(random_state):
            seed(random_state)
        
    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.

        Returns
        -------
        self : object

        """
        self._initialize_weights(X.shape[1])
        self.cost_ = []
        for i in range(self.n_iter):
            if self.shuffle:
                X, y = self._shuffle(X, y)
            cost = []
            for xi, target in zip(X, y):
                cost.append(self._update_weights(xi, target))
            avg_cost = sum(cost) / len(y)
            self.cost_.append(avg_cost)
        return self

    def partial_fit(self, X, y):
        """Fit training data without reinitializing the weights"""
        if not self.w_initialized:
            self._initialize_weights(X.shape[1])
        if y.ravel().shape[0] > 1:
            for xi, target in zip(X, y):
                self._update_weights(xi, target)
        else:
            self._update_weights(X, y)
        return self

    def _shuffle(self, X, y):
        """Shuffle training data"""
        r = np.random.permutation(len(y))
        return X[r], y[r]
    
    def _initialize_weights(self, m):
        """Initialize weights to zeros"""
        self.w_ = np.zeros(1 + m)
        self.w_initialized = True
        
    def _update_weights(self, xi, target):
        """Apply Adaline learning rule to update the weights"""
        output = self.net_input(xi)
        error = (target - output)
        self.w_[1:] += self.eta * xi.dot(error)
        self.w_[0] += self.eta * error
        cost = 0.5 * error**2
        return cost
    
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, X):
        """Compute linear activation"""
        return self.net_input(X)

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.activation(X) >= 0.0, 1, -1)

In [66]:
# Converging, finally
ada = AdalineSGD(n_iter=15, eta=0.01, random_state=1)
ada.fit(X_std, y)

plot_decision_regions(X_std, y, classifier=ada)
plt.title('Adaline - Stochastic Gradient Descent')
plt.xlabel('sepal length [standardized]')
plt.ylabel('petal length [standardized]')
plt.legend(loc='upper left')

plt.tight_layout()
#plt.savefig('./adaline_4.png', dpi=300)
plt.show()

plt.plot(range(1, len(ada.cost_) + 1), ada.cost_, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Average Cost')

plt.tight_layout()
# plt.savefig('./adaline_5.png', dpi=300)
plt.show()